**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Nov 11 08:11:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'(공유)조별활동'
 최종프로젝트
 0930.pdf
'최종프로젝트 (1)'
 1007.pdf
'1022 물화특2.mp3'
'울프 콜 한글자막 1080p 토렌트.torrent'
 123123.jpg
 1541910968438.jpg
 160481207346406.jpg
 레포트-1.docx
'제목 없는 문서 (1).gdoc'
 20190423170408596.pdf
'2019 언팩 오프닝.mp4'
 2019.pdf
'제목 없는 문서 (2).gdoc'
'(신청서)서울시 재난심리회복지원센터 활동가 신청서(서식) - 2페이지.jpg'
'전학영 2차과제.pptx'
 3조
 35-354450_amd-e2-hd-png-download.png
'제목 없는 문서 (3).gdoc'
'제목 없는 문서 (4).gdoc'
 재료화학-9장-무기고분자.pdf
 coco.names
'Colab Notebooks'
 darknet
 Data
 김범진-초록.docx
'제목 없는 문서.gdoc'
'뚜또지제 리슨스테이지.gsheet'
'여름방학 출사계획.gsheet'
'제목없는 스프레드시트.gsheet'
'제목 없는 프레젠테이션.gslides'
 논문.hwp
 김범진-졸업논문.hwp
'나일론의 물성.hwp'


**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15339, done.
remote: Counting objects: 100% (15339/15339), done.
remote: Compressing objects: 100% (4698/4698), done.
remote: Total 15339 (delta 10317), reused 15334 (delta 10312), pack-reused 0
Receiving objects: 100% (15339/15339), 13.99 MiB | 25.53 MiB/s, done.
Resolving deltas: 100% (10317/10317), done.


# 새 섹션

**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp /content/gdrive/MyDrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [8]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /content/drive/MyDrive/yolov3/data/obj.names data/obj.names
!cp /content/drive/MyDrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

cp: cannot stat '/content/drive/MyDrive/yolov3/data/obj.names': Transport endpoint is not connected
cp: cannot stat '/content/drive/MyDrive/yolov3/data/obj.data': Transport endpoint is not connected


In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-11-11 08:14:17--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  66.3MB/s    in 2.3s    

2021-11-11 08:14:19 (66.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /content/gdrive/MyDrive/yolov3/images.zip -d data/obj

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: data/obj/images/S2-N1803M02088.jpg  
 extracting: data/obj/images/S2-N1803M02088.txt  
  inflating: data/obj/images/S2-N1803M02090.jpg  
 extracting: data/obj/images/S2-N1803M02090.txt  
  inflating: data/obj/images/S2-N1803M02091.jpg  
 extracting: data/obj/images/S2-N1803M02091.txt  
  inflating: data/obj/images/S2-N1803M02092.jpg  
 extracting: data/obj/images/S2-N1803M02092.txt  
  inflating: data/obj/images/S2-N1803M02093.jpg  
 extracting: data/obj/images/S2-N1803M02093.txt  
  inflating: data/obj/images/S2-N1803M02094.jpg  
 extracting: data/obj/images/S2-N1803M02094.txt  
 extracting: data/obj/images/S2-N1803M02095.jpg  
 extracting: data/obj/images/S2-N1803M02095.txt  
 extracting: data/obj/images/S2-N1803M02096.jpg  
  inflating: data/obj/images/S2-N1803M02096.txt  
 extracting: data/obj/images/S2-N1803M02097.jpg  
  inflating: data/obj/images/S2-N1803M02097.txt  
 extracting: data/obj/images/S2-N1803M02098.jpg  
 extracting: d

In [11]:
import glob
images_list = glob.glob("data/obj/images/*.png")
#images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/images/hard_hat_workers3248.png', 'data/obj/images/hard_hat_workers2221.png', 'data/obj/images/hard_hat_workers867.png', 'data/obj/images/hard_hat_workers2544.png', 'data/obj/images/hard_hat_workers1779.png', 'data/obj/images/hard_hat_workers4697.png', 'data/obj/images/hard_hat_workers845.png', 'data/obj/images/hard_hat_workers4457.png', 'data/obj/images/hard_hat_workers1076.png', 'data/obj/images/hard_hat_workers731.png', 'data/obj/images/hard_hat_workers2602.png', 'data/obj/images/hard_hat_workers3442.png', 'data/obj/images/hard_hat_workers1212.png', 'data/obj/images/hard_hat_workers2517.png', 'data/obj/images/hard_hat_workers3013.png', 'data/obj/images/hard_hat_workers1973.png', 'data/obj/images/hard_hat_workers3334.png', 'data/obj/images/hard_hat_workers1448.png', 'data/obj/images/hard_hat_workers1024.png', 'data/obj/images/hard_hat_workers869.png', 'data/obj/images/hard_hat_workers3697.png', 'data/obj/images/hard_hat_workers4328.png', 'data/obj/images/hard_hat_workers46

In [12]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [15]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.852686), count: 14, class_loss = 0.416294, iou_loss = 0.270079, total_loss = 0.686373 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.732671), count: 12, class_loss = 0.521322, iou_loss = 0.823218, total_loss = 1.344540 
 total_bbox = 192240, rewritten_bbox = 0.088431 % 

 5535: 0.677710, 0.976745 avg loss, 0.000010 rate, 0.261243 seconds, 44280 images, 0.072012 hours left
Loaded: 0.000085 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.880349), count: 2, class_loss = 0.009795, iou_loss = 0.015236, total_loss = 0.025031 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.841409), count: 20, class_loss = 1.217617, iou_loss = 0.193842, total_loss = 1.411459 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.717111), count: 11, class

In [14]:
# Start the training 2
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
train: Using default 'data/train.txt'
valid: Using default 'data/train.txt'
backup: Using default '/backup/'
yolov3_training
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 8, batch = 8, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 1